<a href="https://colab.research.google.com/github/AeonRosyad/SpanBERT_NLP/blob/main/SpanBERTNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparation

In [ ]:
!python -m spacy download en_core_web_sm
!pip install transformers
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 22.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import numpy as np
import spacy
import ast
from torch.utils.data import Dataset
import torch
from transformers import Adafactor, get_linear_schedule_with_warmup, AdamW
from typing import Tuple
from transformers import Trainer, TrainingArguments
from transformers import BertTokenizerFast, BertForTokenClassification
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
from itertools import groupby
from operator import itemgetter
import ast
from numpy import exp
import importlib, sys
from nltk.tokenize import WhitespaceTokenizer
from google.colab import drive

## Penggabungan Dataset DAMIR & DOSSPIRE

In [ ]:
# Membaca dataset
df = pd.read_excel('/content/DAMIR.xlsx')
df_2 = pd.read_csv('/content/DOSSPRE 1.0Original.csv', encoding='latin1')
df_3 = pd.read_csv('/content/software_requirements_extended.csv', encoding='latin1')
data_1 = df['Context']
data_2 = df_2['Requirement']
data_3 = df_3['Requirement']

In [ ]:
# Menambahkan kolom 'Source' untuk membedakan sumber data
data_1 = data_1.rename_axis('Text').reset_index()
data_1['Source'] = 'DAMIR'

In [ ]:
data_2 = data_2.rename_axis('Text').reset_index()
data_2['Source'] = 'DOSSPRE'

In [ ]:
data_3 = data_3.rename_axis('Text').reset_index()
data_3['Source'] = 'KAGGLETYPE'

In [ ]:
# Langkah 1: Gabungkan dataset
df = pd.concat([data_1, data_2, data_3], axis=0, ignore_index=True)

# Langkah 2: Simpan hasil penggabungan ke dalam CSV
df.to_csv('dataset.csv', index=False)

dft = pd.read_csv('dataset.csv')

In [ ]:
# Mengisi nilai NaN di kolom 'Requirement' dengan nilai dari kolom 'Context'
dft['Requirement'].fillna(dft['Context'], inplace=True)
dft = dft.drop(columns=['Context', 'Text'])
dft


,Source,Requirement
0,DAMIR,"Under the Halifax-class WPC-W and WPC-E, DND w..."
1,DAMIR,"Under the Halifax-class WPC-W and WPC-E, DND w..."
2,DAMIR,"Under the Halifax-class WPC-W and WPC-E, DND w..."
3,DAMIR,"Under the Halifax-class WPC-W and WPC-E, DND w..."
4,DAMIR,"Under the Halifax-class WPC-W and WPC-E, DND w..."
...,...,...
7793,KAGGLETYPE,There will be a designated phone number that u...
7794,KAGGLETYPE,Texts sent to that number will be sent to the ...
7795,KAGGLETYPE,"If a question is not understood by our API, th..."
7796,KAGGLETYPE,Upon the USB being plugged in the system shall...


## Data Cleaning

In [ ]:
# Mengahpus Duplikat
data = dft.drop_duplicates()
data = data.reset_index(drop=True)

# Menghapus baris dengan missing values
data = data.dropna()
data

,Source,Requirement
0,DAMIR,"Under the Halifax-class WPC-W and WPC-E, DND w..."
1,DAMIR,Canada will use other organizations to support...
2,DAMIR,The ship readiness levels and expected annual ...
3,DAMIR,HMC Ships on the West Coast are assigned to Ma...
4,DAMIR,The MARLANT and MARPAC Deputy Chiefs of Staff ...
...,...,...
2825,KAGGLETYPE,There will be a designated phone number that u...
2826,KAGGLETYPE,Texts sent to that number will be sent to the ...
2827,KAGGLETYPE,"If a question is not understood by our API, th..."
2828,KAGGLETYPE,Upon the USB being plugged in the system shall...


#Deteksi Anaforis Kata Ganti Ambigu

In [ ]:
df = data.rename(columns={'Requirement': 'Context'})
exampleData = df['Context']
exampleData

0       Under the Halifax-class WPC-W and WPC-E, DND w...
1       Canada will use other organizations to support...
2       The ship readiness levels and expected annual ...
3       HMC Ships on the West Coast are assigned to Ma...
4       The MARLANT and MARPAC Deputy Chiefs of Staff ...
                              ...                        
2825    There will be a designated phone number that u...
2826    Texts sent to that number will be sent to the ...
2827    If a question is not understood by our API, th...
2828    Upon the USB being plugged in the system shall...
2829    The system shall be able to handle 1000 custom...
Name: Context, Length: 2830, dtype: object

## Processing data

In [ ]:
nlp = spacy.load("en_core_web_sm")

def applynlp(string,nlp):
    tr=np.nan
    try:
        tr=nlp(string)
    except:
        print(string)
    return tr

exampleData= exampleData.apply(lambda x: applynlp(x,nlp))
exampleData

0       (Under, the, Halifax, -, class, WPC, -, W, and...
1       (Canada, will, use, other, organizations, to, ...
2       (The, ship, readiness, levels, and, expected, ...
3       (HMC, Ships, on, the, West, Coast, are, assign...
4       (The, MARLANT, and, MARPAC, Deputy, Chiefs, of...
                              ...                        
2825    (There, will, be, a, designated, phone, number...
2826    (Texts, sent, to, that, number, will, be, sent...
2827    (If, a, question, is, not, understood, by, our...
2828    (Upon, the, USB, being, plugged, in, the, syst...
2829    (The, system, shall, be, able, to, handle, 100...
Name: Context, Length: 2830, dtype: object

In [ ]:
nlp = spacy.load("en_core_web_sm")
pronouns=["I","me","my","mine","myself","you","your","yours","yourself","he","him","his","himself","she","her","hers","herself","it","its","itself","we","us","our","ours","ourselves","they","them","their","theirs","themselves"]
li=[]
i,j=0,0
ids=[]

def findPronouns(sent, pronouns, nlp):
    # Menganalisis kalimat menjadi objek token
    doc = nlp(sent)
    tokens = []

    for t in doc:
        if "PRP" in t.tag_ and t.text.lower() in pronouns and t not in tokens:
            tokens.append(t)

    return tokens

def getNPs(sent,p,include_nouns=False):
    nps=[]
    npstr=[]
    chunks = list(sent.noun_chunks)
    for i in range(len(chunks)):
        np=chunks[i]
        if np.end<=p.i:
            if len(np)==1:
                if np[0].pos_ not in ["NOUN","PROPN"]:
                    continue
            if np.text.lower() in npstr:
                for x in nps:
                    if x.text.lower() == np.text.lower():
                        nps.remove(x)
                npstr.remove(np.text.lower())
            nps.append(np)
            npstr.append(np.text.lower())
            if i < len(chunks)-1:
                np1=chunks[i+1]
                if np1.start-np.end==1:
                    if sent.doc[np.end].tag_=="CC":
                        newnp = sent.doc[np.start:np1.end]
                        if newnp.text.lower() in npstr:
                            for x in nps:
                                if x.text.lower() == newnp.text.lower():
                                    nps.remove(x)
                            npstr.remove(newnp.text.lower())
                        nps.append(newnp)
                        npstr.append(newnp.text.lower())

    if include_nouns:
        for t in sent:
            if t.i<p.i and "subj" in t.dep_ and t.pos_=="NOUN": # to revisit
                if t.text.lower() in npstr:
                    for x in nps:
                        if x.text.lower() == t.text.lower():
                            nps.remove(x)
                    npstr.remove(t.text.lower())
                npstr.append(t.text.lower())
                nps.append(sent[t.i:t.i+1])
    return nps

for context in exampleData:
    pronoun = findPronouns(context, pronouns, nlp)  # Mengubah loop untuk hanya satu kata ganti
    if pronoun:
        pronoun = pronoun[0]  # Mengambil hanya satu kata ganti
        Id = str(i)+"-"+pronoun.text+"-"+str(j)
        while Id in ids:
            j+=1
            Id = str(i)+"-"+pronoun.text+"-"+str(j)

        candidateAntecedent = getNPs(context, pronoun)  # Mengambil kandidat antecedent
        if candidateAntecedent:
            candidateAntecedent = candidateAntecedent[0]  # Mengambil hanya satu kandidat antecedent

            li.append([Id, context, pronoun, pronoun.i, candidateAntecedent])
            ids.append(Id)
        else:  # Jika tidak ada kandidat antecedent
            li.append([Id, context, pronoun, pronoun.i, None])
            ids.append(Id)
    else:  # Jika tidak ada kata ganti dalam kalimat
        li.append([str(i) + "-NoPronoun-" + str(j), context, None, None, None])
        j += 1

    i += 1

exampleDataProcessed = pd.DataFrame(li, columns=["Id","Context","Pronoun","Position","Candidate Antecedent"])
exampleDataProcessed


,Id,Context,Pronoun,Position,Candidate Antecedent
0,0-its-0,"(Under, the, Halifax, -, class, WPC, -, W, and...",its,77.0,"(the, Halifax, -, class, WPC, -, W, and, WPC, ..."
1,1-their-0,"(Canada, will, use, other, organizations, to, ...",their,60.0,(Canada)
2,2-its-0,"(The, ship, readiness, levels, and, expected, ...",its,32.0,"(The, ship, readiness, levels)"
3,3-their-0,"(HMC, Ships, on, the, West, Coast, are, assign...",their,30.0,"(HMC, Ships)"
4,4-their-0,"(The, MARLANT, and, MARPAC, Deputy, Chiefs, of...",their,14.0,"(The, MARLANT)"
...,...,...,...,...,...
2825,2825-NoPronoun-1844,"(There, will, be, a, designated, phone, number...",None,NaN,None
2826,2826-NoPronoun-1845,"(Texts, sent, to, that, number, will, be, sent...",None,NaN,None
2827,2827-our-1846,"(If, a, question, is, not, understood, by, our...",our,7.0,"(a, question)"
2828,2828-NoPronoun-1846,"(Upon, the, USB, being, plugged, in, the, syst...",None,NaN,None


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
fast_tokenizer = BertTokenizerFast.from_pretrained('SpanBERT/spanbert-base-cased')
model_path_nlp = "/content/drive/MyDrive/SpanBERT-NLPv21.8.10"
model_path_re = "/content/drive/MyDrive/SpanBERT-REv21.9.01"

nlpmodel = BertForTokenClassification.from_pretrained(model_path_nlp)
remodel = BertForTokenClassification.from_pretrained(model_path_re)

OSError: Can't load tokenizer for 'SpanBERT/spanbert-base-cased'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'SpanBERT/spanbert-base-cased' is the correct path to a directory containing all relevant files for a BertTokenizerFast tokenizer.

In [ ]:
def get_tokenclassification_annotation(item, fast_tokenizer, train=True, debug=False,max_length=128):
  text = item['context']+" [SEP] "+item['pronoun']
  tokens_info = fast_tokenizer(text, padding='max_length', max_length=max_length, truncation=True, return_offsets_mapping = True)
  # padding='max_length', max_length=max_length, truncation=True,
  if debug:
    print(f'Original Text:\n\t {text}')
  if train:
    spans = transform_strtab_inttab(item['schars'])
    chunks = compute_chunks(spans)
    if debug:
      print("Antecedent text")
      for chunk in chunks:
        print('\t',text[chunk[0]:chunk[-1]])
    tokenized_text = fast_tokenizer.convert_ids_to_tokens(tokens_info['input_ids'])
    if debug:
      print(f'Tokenized Text:\n\t {tokenized_text}')
    offsets_mapping = tokens_info['offset_mapping']
    annot = []
    for k in spans:
      for x, (j, v) in enumerate(offsets_mapping[1:-1]):
        if j <= k and k < v  :
          annot.append(x)
    annot = set(annot)
    annotation = [0 for x in  offsets_mapping[1:-1]]
    if debug:
      print("Antecedent words")
    for x in annot:
      annotation[x] = 1
      if debug:
        print('\t',tokenized_text[1:-1][x], "||||index: ", x)
    return {'input_ids':tokens_info['input_ids'],
            'labels':[0] + annotation + [0],
            'attention_mask': tokens_info['attention_mask']
    }
  return {'input_ids':tokens_info['input_ids'],
            'attention_mask': tokens_info['attention_mask']
    }

def transform_strtab_inttab(s):
  s=str(s)
  return [int(x) for x in s[1:-1].split(',') if len(x)>0]

def compute_chunks(tab):
  init_chunck  = 0
  chunks = []
  for i, k in enumerate(tab[:-1]):
    if k+1 != tab[i+1]:
      chunks.append(tab[init_chunck: i+1])
      init_chunck = i+1
  chunks.append(tab[init_chunck:])
  return chunks


class SpanDetectionData(Dataset):
  def __init__(self, df, tokenizer, train=True):
    self.df = df
    self.tokenizer = tokenizer
    self.train = train

  def __len__(self,):
    return len(self.df)

  def __getitem__(self, idx):
    item = get_tokenclassification_annotation(self.df.iloc[idx], self.tokenizer, self.train)
    return item

test=[]
test_no_pronoun=[]  # DataFrame untuk kalimat tanpa kata ganti

for Id in exampleDataProcessed.Id.unique():
    c = exampleDataProcessed[exampleDataProcessed.Id==Id].Context.unique()[0]
    pronoun = exampleDataProcessed[exampleDataProcessed.Id==Id].Pronoun.unique()[0]

    if c is not None and pronoun is not None:  # Memeriksa apakah c dan pronoun tidak None
        hashedpronoun=pronoun.text+"#1"
        hashedcontext=c[:pronoun.i].text+" "+hashedpronoun+" "+c[pronoun.i+1:].text
        test.append([Id,hashedcontext,hashedpronoun])
    else:
        hashedcontext = ""  # Kalimat tanpa kata ganti akan diabaikan
        test_no_pronoun.append([Id, c, pronoun])  # Menyimpan kalimat tanpa kata ganti

testdf=pd.DataFrame(test,columns=["Id","context","pronoun"])
test_data = SpanDetectionData(testdf, fast_tokenizer,train=False)

testdf_no_pronoun = pd.DataFrame(test_no_pronoun, columns=["Id","context","pronoun"])
test_data_no_pronoun = SpanDetectionData(testdf_no_pronoun, fast_tokenizer, train=False)
testdf_no_pronoun['context'] = testdf_no_pronoun['context'].apply(lambda x: x.text if x is not None else "")


### SpanBERT Solution


In [ ]:
for param in nlpmodel.base_model.parameters():    # semua parameter dalam model nlpmodel menjadi False
    param.requires_grad = False       # parameter-parameter ini tidak akan mengalami pembelajaran (learning) selama proses training

In [ ]:
# Mengatur Argumentasi Pelatihan
training_args = TrainingArguments(             # parameter dan konfigurasi yang diperlukan untuk melatih model
    output_dir='./results',                    # Direktori tempat hasil training akan disimpan.
    eval_steps=200,
    logging_steps=200,                         # frekuensi logging selama training (setiap 200 iterasi).
    save_steps=200,                            # Frekuensi penyimpanan checkpoint selama training (setiap 200 iterasi).
    save_total_limit=5,                        # Jumlah checkpoint yang akan disimpan secara keseluruhan (maksimal 5).
    num_train_epochs=4,                        # Jumlah epoch yang akan dilatih (4 epoch)
    # learning_rate=2e-5,                      # Tingkat pembelajaran (learning rate) yang digunakan (2e-5).
    per_device_train_batch_size=32,            # Jumlah sampel yang digunakan per batch selama training (32 sampel per perangkat).
    per_device_eval_batch_size=32,             # Jumlah sampel yang digunakan per batch selama evaluasi (32 sampel per perangkat).
    warmup_steps=500,                          # Jumlah langkah "pemanasan" awal selama training.
    weight_decay=0.01,                         # Pengaturan penalti untuk regularisasi L2.
    logging_dir='./logs',                      # Direktori tempat log dari training akan disimpan.
    report_to='tensorboard',
    run_name='span_detection'
)

# Inisiasi Trainer
nlp_trainer = Trainer(                         # inisialisasi sebagai Trainer yang akan digunakan untuk melatih model
    model=nlpmodel,                            # Model yang akan dilatih
    args=training_args,                        # konfigurasi pelatihan
)


In [ ]:
def continuousSubArrays(L):
  sas=[]
  sa=[]
  for i,j in zip(range(0,len(L)-1),range(1,len(L))):
          if L[i]==L[j]-1 or L[i]==L[j]-2:
              if L[i] not in sa:
                  sa.append(L[i])
              if L[j] not in sa:
                  sa.append(L[j])
          else:
              if sa:
                  sas.append(sa)
                  sa=[]
  if sa:
      sas.append(sa)
  return sas

def softmax(vector):
  li=[]
  for element in vector:
    se=exp(element).sum()
    ne=exp(element)/se
    li.append(ne)
  return np.array(li)

def predict_spans(truncated_predictions, cdf,fast_tokenizer,T=0):
  predited_spans = []
  mean=-sys.maxsize
  for i in range(len(cdf)):
    span=[]
    nmean=-sys.maxsize-1
    aboveT=0
    for x in continuousSubArrays(np.where(np.argmax(truncated_predictions[i],1)==1)[0]):
        if softmax(truncated_predictions[i])[:,1][x].mean()>nmean:
            nmean=softmax(truncated_predictions[i])[:,1][x].mean()
            if nmean>T:
              aboveT+=1
            span=(np.array(x, dtype=np.int64),)
    #nmean=softmax(truncated_predictions[i])[:,1][span].mean()
    if T!=0:
        if nmean<T:
            predited_spans.append([])
            continue
    if aboveT>1:
      predited_spans.append([])
      continue
    text = cdf.iloc[i]['context']
    #print(text)
    #discminative_preds = np.argmax(truncated_predictions[i],1)
    #print(len(discminative_preds))
    one_idx = span#np.where(discminative_preds == 1)
    one_idx = one_idx[0]
    tokens_info = fast_tokenizer(text, return_offsets_mapping=True)
    # print(len(tokens_info['input_ids']))
    # print(len(tokens_info['offset_mapping']))
    #print("************************")
    yoyo = [tokens_info['offset_mapping'][k] for k in one_idx if k<len(tokens_info['offset_mapping'])]
    pred = []
    for p in yoyo:
      pred += list(range(p[0],p[1]))
    predited_spans.append(pred)
  return predited_spans

def processPred(predictions,train_data,df,fast_tokenizer,T=0.9):
    truncated_predictions = []
    for k in range(len(train_data)):
      inputs_ids = train_data[k]['input_ids']
      truncated_predictions.append(predictions.predictions[k][:len(inputs_ids)])
    predicted_spans = predict_spans(truncated_predictions, df,fast_tokenizer,T)
    return truncated_predictions,predicted_spans

# Melakukan Prediksi dengan Model
nlp_predictions = nlp_trainer.predict(test_data)

# Proses Prediksi:
ttruncated_predictions,tpredicted_spans=processPred(nlp_predictions,test_data,testdf,fast_tokenizer,T=0.9)    # Hasil prediksi dari model diproses menggunakan fungsi processPred

In [ ]:
def findspans(s,ids):
    spans=[]
    lastspan=""
    previ=-1
    for i in ids:
      i=int(i)
      if previ==-1 or i==previ+1:
          lastspan+=s[i]
          previ=i
          continue
      elif i==previ+2:
          lastspan+=" "+s[i]
          previ=i
          continue
      else:
          if lastspan not in spans:
              if lastspan:
                spans.append(lastspan)
          lastspan=s[i]
          previ=i
          continue
    if lastspan not in spans:
        if lastspan:
          spans.append(lastspan)
    return spans

spans=[]
for i,j in zip(testdf.index, tpredicted_spans):
    spans.append(findspans(testdf.context[i],j))
testdf['Resolution']=spans
testdf['Detection']=testdf['Resolution'].apply(lambda x: "Unambiguous" if len(x)!=0 else "Ambiguous")
testdf


##Hasil Penanganan Ambiguitas

In [ ]:
import pandas as pd

# Buat DataFrame baru untuk menyimpan hasil
output_data = []

for i in testdf.index:
    id = testdf['Id'][i]
    context = testdf['context'][i]
    pronoun = testdf['pronoun'][i]
    antecedent = testdf['Resolution'][i]
    detected_as = testdf['Detection'][i]

    # Tambahkan baris baru ke DataFrame
    output_data.append({'Id': id, 'Context': context, 'Pronouns': pronoun, 'Antecedent': antecedent, 'Pronouns_Ambiguous': detected_as})

# Buat DataFrame dari list data
output_df = pd.DataFrame(output_data)
output_df['Context'] = output_df['Context'].str.replace('#1', '', regex=False)

# Cetak DataFrame
output_df


In [ ]:
# Langkah 1: Gabungkan dataset
df = pd.concat([output_df, testdf_no_pronoun], axis=0, ignore_index=True)

# Langkah 2: Simpan hasil penggabungan ke dalam CSV
df.to_csv('pronouns_ambiguous.csv', index=False)

pronoun_ambigu = pd.read_csv('pronouns_ambiguous.csv')

In [ ]:
# Mengisi nilai NaN di kolom 'Requirement' dengan nilai dari kolom 'Context'
pronoun_ambigu['Context'].fillna(pronoun_ambigu['context'], inplace=True)
pronoun_ambigu = pronoun_ambigu.drop(columns=['context','pronoun'])
pronoun_ambigu = pronoun_ambigu.reset_index(drop=True)

In [ ]:
import pandas as pd

# Misalkan Anda memiliki DataFrame 'data' yang berisi data 'Requirement' dan 'Source'
# 'pronoun_ambigu' adalah DataFrame yang sudah memiliki kolom 'Context' dan 'Pronouns_Ambiguous'

def find_matching_source(context, dft):
    # Memisahkan kata-kata dari 'Context' menjadi token
    context_tokens = set(context.split())

    # Mencari Requirement dengan token terbanyak yang cocok
    best_matching_requirement = None
    max_matching_tokens = 0

    for requirement in dft['Requirement']:
        requirement_tokens = set(requirement.split())
        matching_tokens = len(context_tokens.intersection(requirement_tokens))

        # Menyimpan nilai 'Source' jika memiliki token yang lebih banyak cocok
        if matching_tokens > max_matching_tokens:
            max_matching_tokens = matching_tokens
            best_matching_requirement = requirement

    # Mengambil nilai 'Source' berdasarkan Requirement dengan token terbanyak yang cocok
    if best_matching_requirement:
        matching_sources = dft[dft['Requirement'] == best_matching_requirement]['Source']
        return matching_sources.mode()[0] if len(matching_sources) > 0 else None
    else:
        return None

pronoun_ambigu['Source'] = pronoun_ambigu['Context'].apply(lambda context: find_matching_source(context, dft))

# Menandai kalimat yang tidak memiliki Pronouns sebagai "Unambiguous"
pronoun_ambigu.loc[pronoun_ambigu['Pronouns'].isna(), 'Pronouns_Ambiguous'] = 'Unambiguous'

# Mengacak urutan baris pada DataFrame 'pronoun_ambigu'
pronoun_ambigu = pronoun_ambigu.sample(frac=1).reset_index(drop=True)

# Menampilkan DataFrame 'pronoun_ambigu' setelah pemrosesan
pronoun_ambigu


In [ ]:
pronoun_ambigu = pronoun_ambigu.reindex(columns=['Id', 'Source', 'Context', 'Pronouns', 'Antecedent', 'Pronouns_Ambiguous'])
pronoun_ambigu

#Deteksi (Kata Sifat ambigu, Kata Keterangan ambigu) dan Frasa Komparatif

In [ ]:
data = pronoun_ambigu
data = data.drop(columns='Id')
data

In [ ]:
# Load model bahasa Inggris dari spaCy
nlp = spacy.load("en_core_web_sm")

# Fungsi untuk menandai part-of-speech dan mendeteksi ambiguitas
def mark_pos_and_detect_ambiguity(text):
    doc = nlp(text)
    marked_text = ""

    for i, token in enumerate(doc):
        if token.pos_ == "ADJ" and token.dep_ == 'amod':
            marked_text += f"[{token.text}] "

        elif token.pos_ == "ADV":
            marked_text += f"[{token.text}] "

        else:
            if '#1' in token.text:
                marked_text += token.text.replace('#1', '') + " "
            else:
                marked_text += token.text + " "
    return marked_text.strip()

# Tambahkan kolom-kolom baru ke DataFrame
data["Marked_Text"] = data['Context'].apply(mark_pos_and_detect_ambiguity)

# Simpan hasilnya kembali ke CSV jika diperlukan
data.to_csv("data_hasil.csv", index=False)  # Gantilah "data_hasil.csv" sesuai kebutuhan Anda

# Baca kembali data hasil
data_hasil = pd.read_csv('data_hasil.csv', encoding='latin1')
data_hasil


In [ ]:
import re

# Fungsi untuk memeriksa apakah ada dependensi 'nummod'
def ambil_kalimat_dalam_kurung_siku(teks):
    return re.findall(r'\[([^\]]+)\]', teks)

def generate_interpretation(text):
    doc = nlp(text)
    interpretation = []

    for token in doc:
        # Lemmatisasi token
        kata_dasar = token.lemma_
        interpretation.append((kata_dasar, token.dep_, token.head.text))

    return interpretation

def check_nummod_dependency(text):
    doc = nlp(text)
    kata_dalam_kurung_siku = ambil_kalimat_dalam_kurung_siku(text)

    if not kata_dalam_kurung_siku:
        return 'Unambiguous'

    interpretation = generate_interpretation(text)
    for token in doc:
        if token.dep_ == 'nummod':
            return 'Unambiguous'

    return 'Ambiguous'

# Tambahkan kolom baru ke DataFrame
data_hasil['ADJ_ADV_Ambiguous'] = data_hasil['Marked_Text'].apply(check_nummod_dependency)
data_hasil


In [ ]:
# Load model bahasa Inggris dari spaCy
nlp = spacy.load("en_core_web_sm")

# Fungsi untuk menandai part-of-speech dan mendeteksi ambiguitas
def mark_pos_and_detect_ambiguity(text):
    doc = nlp(text)
    comparative_phrases = []

    for i, token in enumerate(doc):
        if i > 0 and doc[i-1].text.lower() in ["more", "less", ]:
            # Cek apakah token sebelumnya adalah "more" atau "less"
            comparative_phrases.append(f"{doc[i-1].text} {token.text}")

        elif token.pos_ == "ADJ" and token.text.endswith("er") and token.text != 'other':
            # Cek apakah kata sifat berakhiran "er"
            comparative_phrases.append(token.text)

    return comparative_phrases

# Tambahkan kolom-kolom baru ke DataFrame
data_hasil["Comparative_Phrases"] = data_hasil['Context'].apply(mark_pos_and_detect_ambiguity)
data_hasil

In [ ]:
import spacy
import pandas as pd

# Load model bahasa Inggris dari spaCy
nlp = spacy.load("en_core_web_sm")

# Fungsi untuk menandai part-of-speech dan mendeteksi ambiguitas
def mark_pos_and_detect_ambiguity(text):
    doc = nlp(text)
    comparative_phrases = []

    for i, token in enumerate(doc):
        if i > 0 and doc[i-1].text.lower() in ["more", "less"]:
            # Cek apakah token sebelumnya adalah "more" atau "less"
            comparative_phrases.append(f"{doc[i-1].text} {token.text}")

        elif token.pos_ == "ADJ" and token.text.endswith("er") and token.text != 'other':
            # Cek apakah kata sifat berakhiran "er"
            comparative_phrases.append(token.text)

    if len(comparative_phrases) > 0:
        # Jika terdapat comparative phrase, tetapi cek apakah ada nummod
        has_nummod = any([token.dep_ == 'nummod' and token.pos_ == 'NUM' for token in doc])
        if has_nummod:
            return 'Unambiguous'
        else:
            return 'Ambiguous'
    else:
        return 'Unambiguous'

# Tambahkan kolom baru ke DataFrame untuk label
data_hasil['Comparative_Phrase_Ambiguous'] = data_hasil['Context'].apply(mark_pos_and_detect_ambiguity)
data_hasil


In [ ]:
# Fungsi untuk mengganti nilai 'Ambiguous' dengan 1 dan 'Unambiguous' dengan 0
def replace_labels(value):
    if value == 'Ambiguous':
        return 1
    elif value == 'Unambiguous':
        return 0
    else:
        return value

# Mengaplikasikan fungsi
data_hasil['Pronouns_Ambiguous'] = data_hasil['Pronouns_Ambiguous'].apply(replace_labels)
data_hasil['ADJ_ADV_Ambiguous'] = data_hasil['ADJ_ADV_Ambiguous'].apply(replace_labels)
data_hasil['Comparative_Phrase_Ambiguous'] = data_hasil['Comparative_Phrase_Ambiguous'].apply(replace_labels)
data_hasil

In [ ]:
# Fungsi untuk memberi label berdasarkan kondisi yang disebutkan
def label_ambiguous(row):
    if row['Pronouns_Ambiguous'] == 1 or row['ADJ_ADV_Ambiguous'] == 1 or row['Comparative_Phrase_Ambiguous'] == 1:
        return 1
    else:
        return 0

# Menambahkan kolom baru 'Label'
data_hasil['Label'] = data_hasil.apply(label_ambiguous, axis=1)

# Menampilkan data setelah penambahan kolom baru
print(data_hasil[['Pronouns_Ambiguous', 'ADJ_ADV_Ambiguous', 'Comparative_Phrase_Ambiguous', 'Label']])


# Analisis Data

## Analisis frekuensi istilah ambigu

### DAMIR

In [ ]:
# Filter dataset untuk mencakup hanya entri 'Source' dan memiliki label ambigu pada kolom Istilah Ambigu
ambiguous_damir_data = data_hasil[(data_hasil['Source'] == 'DAMIR') & (data_hasil['Pronouns_Ambiguous'] == 1)]
ambiguous_damir_data_2 = data_hasil[(data_hasil['Source'] == 'DAMIR') & (data_hasil['ADJ_ADV_Ambiguous'] == 1)]
ambiguous_damir_data_3 = data_hasil[(data_hasil['Source'] == 'DAMIR') & (data_hasil['Comparative_Phrase_Ambiguous'] == 1)]
ambiguous_label_damir = data_hasil[(data_hasil['Source'] == 'DAMIR') & (data_hasil['Label'] == 1)]

# Hitung jumlah baris yang memiliki 'Source' DAMIR dan label ambigu pada kolom 'Pronouns_Ambiguous'
total_ambiguous_damir = ambiguous_damir_data.shape[0]
total_ambiguous_damir_2 = ambiguous_damir_data_2.shape[0]
total_ambiguous_damir_3 = ambiguous_damir_data_3.shape[0]
total_ambiguous_label_damir = ambiguous_label_damir.shape[0]

# Hitung total baris yang memiliki 'Source' DAMIR
total_damir_data = data_hasil[data_hasil['Source'] == 'DAMIR'].shape[0]

# Hitung persentase 'Context' yang memiliki label ambigu pada dataset yang memiliki 'Source' DAMIR
percentage_ambiguous_damir = (total_ambiguous_damir / total_damir_data) * 100
percentage_ambiguous_damir_2 = (total_ambiguous_damir_2 / total_damir_data) * 100
percentage_ambiguous_damir_3 = (total_ambiguous_damir_3 / total_damir_data) * 100

print(f"DAMIR (Kata Ganti Ambigu): {percentage_ambiguous_damir:.2f}%")
print(f"DAMIR (Kata Sifat & Keterangan Ambigu): {percentage_ambiguous_damir_2:.2f}%")
print(f"DAMIR (Frasa Komparatif Ambigu): {percentage_ambiguous_damir_3:.2f}%")
print("Total requirements Ambigu pada dataset DAMIR :", total_ambiguous_label_damir)

###DOSSPRE



In [ ]:
# Filter dataset untuk mencakup hanya entri 'Source' dan memiliki label ambigu pada kolom Istilah Ambigu
ambiguous_data_dosspre = data_hasil[(data_hasil['Source'] == 'DOSSPRE') & (data_hasil['Pronouns_Ambiguous'] == 1)]
ambiguous_data_2_dosspre = data_hasil[(data_hasil['Source'] == 'DOSSPRE') & (data_hasil['ADJ_ADV_Ambiguous'] == 1)]
ambiguous_data_3_dosspre = data_hasil[(data_hasil['Source'] == 'DOSSPRE') & (data_hasil['Comparative_Phrase_Ambiguous'] == 1)]
ambiguous_dosspre_label = data_hasil[(data_hasil['Source'] == 'DOSSPRE') & (data_hasil['Label'] == 1)]

# Hitung jumlah baris yang memiliki 'Source' DAMIR dan label ambigu pada kolom 'Pronouns_Ambiguous'
total_ambiguous_dosspre = ambiguous_data_dosspre.shape[0]
total_ambiguous_2_dosspre = ambiguous_data_2_dosspre.shape[0]
total_ambiguous_3_dosspre = ambiguous_data_3_dosspre.shape[0]
total_ambiguous_label_dosspre = ambiguous_dosspre_label.shape[0]

# Hitung total baris yang memiliki 'Source' DAMIR
total_data_dosspre = data_hasil[data_hasil['Source'] == 'DOSSPRE'].shape[0]

# Hitung persentase 'Context' yang memiliki label ambigu pada dataset yang memiliki 'Source' DAMIR
percentage_ambiguous = (total_ambiguous_dosspre / total_data_dosspre) * 100
percentage_ambiguous_2 = (total_ambiguous_2_dosspre / total_data_dosspre) * 100
percentage_ambiguous_3 = (total_ambiguous_3_dosspre / total_data_dosspre) * 100

print(f"DOSSPRE (Kata Ganti Ambigu): {percentage_ambiguous:.2f}%")
print(f"DOSSPRE (Kata Sifat & Keterangan Ambigu): {percentage_ambiguous_2:.2f}%")
print(f"DOSSPRE (Frasa Komparatif Ambigu): {percentage_ambiguous_3:.2f}%")
print("Total requirements Ambigu pada dataset DOSSPRE :", total_ambiguous_label_dosspre)


### Type Requirement (Kaggle)

In [ ]:
# Filter dataset untuk mencakup hanya entri 'Source' dan memiliki label ambigu pada kolom Istilah Ambigu
ambiguous_data = data_hasil[(data_hasil['Source'] == 'KAGGLETYPE') & (data_hasil['Pronouns_Ambiguous'] == 1)]
ambiguous_data_2 = data_hasil[(data_hasil['Source'] == 'KAGGLETYPE') & (data_hasil['ADJ_ADV_Ambiguous'] == 1)]
ambiguous_data_3 = data_hasil[(data_hasil['Source'] == 'KAGGLETYPE') & (data_hasil['Comparative_Phrase_Ambiguous'] == 1)]
ambiguous_type_label = data_hasil[(data_hasil['Source'] == 'KAGGLETYPE') & (data_hasil['Label'] == 1)]


# Hitung jumlah baris yang memiliki 'Source' DAMIR dan label ambigu pada kolom 'Pronouns_Ambiguous'
total_ambiguous = ambiguous_data.shape[0]
total_ambiguous_2 = ambiguous_data_2.shape[0]
total_ambiguous_3 = ambiguous_data_3.shape[0]
total_ambiguous_label_type = ambiguous_type_label.shape[0]

# Hitung total baris yang memiliki 'Source' DAMIR
total_data_kaggle = data_hasil[data_hasil['Source'] == 'KAGGLETYPE'].shape[0]

# Hitung persentase 'Context' yang memiliki label ambigu pada dataset yang memiliki 'Source' DAMIR
percentage_ambiguous = (total_ambiguous / total_data_kaggle) * 100
percentage_ambiguous_2 = (total_ambiguous_2 / total_data_kaggle) * 100
percentage_ambiguous_3 = (total_ambiguous_3 / total_data_kaggle) * 100

print(f"KAGGLETYPE (Kata Ganti Ambigu): {percentage_ambiguous:.2f}%")
print(f"KAGGLETYPE (Kata Sifat & Keterangan Ambigu): {percentage_ambiguous_2:.2f}%")
print(f"KAGGLETYPE (Frasa Komparatif Ambigu): {percentage_ambiguous_3:.2f}%")
print("Total requirements Ambigu pada dataset Kaggle :", total_ambiguous_label_type)


### Seluruh Dataset

In [ ]:
percentage_pronouns_ambiguous = (data_hasil['Pronouns_Ambiguous'].sum() / len(data_hasil)) * 100

percentage_adj_adv_ambiguous = (data_hasil['ADJ_ADV_Ambiguous'].sum() / len(data_hasil)) * 100

percentage_comparative_phrase_ambiguous = (data_hasil['Comparative_Phrase_Ambiguous'].sum() / len(data_hasil)) * 100

print(f"Presentase Pronouns yang berlabel ambigu: {percentage_pronouns_ambiguous:.2f}%")
print(f"Presentase ADJ_ADV yang berlabel ambigu: {percentage_adj_adv_ambiguous:.2f}%")
print(f"Presentase Comparative Phrase yang berlabel ambigu: {percentage_comparative_phrase_ambiguous:.2f}%")


## Analisis kombinasi kemungkinan ambigu pada dataset

In [ ]:
import pandas as pd

# Kombinasi nilai yang akan dihitung
combinations = {
    '111': ['Pronouns_Ambiguous', 'ADJ_ADV_Ambiguous', 'Comparative_Phrase_Ambiguous'],
    '100': ['Pronouns_Ambiguous', 'ADJ_ADV_Ambiguous', 'Comparative_Phrase_Ambiguous'],
    '010': ['Pronouns_Ambiguous', 'ADJ_ADV_Ambiguous', 'Comparative_Phrase_Ambiguous'],
    '001': ['Pronouns_Ambiguous', 'ADJ_ADV_Ambiguous', 'Comparative_Phrase_Ambiguous'],
    '110': ['Pronouns_Ambiguous', 'ADJ_ADV_Ambiguous', 'Comparative_Phrase_Ambiguous'],
    '011': ['Pronouns_Ambiguous', 'ADJ_ADV_Ambiguous', 'Comparative_Phrase_Ambiguous'],
    '101': ['Pronouns_Ambiguous', 'ADJ_ADV_Ambiguous', 'Comparative_Phrase_Ambiguous'],
    '000': ['Pronouns_Ambiguous', 'ADJ_ADV_Ambiguous', 'Comparative_Phrase_Ambiguous']
}

# Inisialisasi variabel untuk menyimpan jumlah kombinasi
combination_counts = {}

# Menghitung jumlah kombinasi nilai
for key, values in combinations.items():
    query = ' & '.join(f"{value} == {int(key[index])}" for index, value in enumerate(values))
    combination_counts[key] = data_hasil.query(query).shape[0]

# Menghitung total data
total_data = data_hasil.shape[0]

# Menghitung presentase setiap kombinasi nilai
percentage_combinations = {key: round((value / total_data) * 100, 2) for key, value in combination_counts.items()}

# Menyimpan hasil presentase ke dalam DataFrame pandas
df = pd.DataFrame(list(percentage_combinations.items()), columns=['Kombinasi', 'Presentase'])

# Menampilkan tabel hasil presentase
df



## Analisis tipe requirement yang kemungkinan besar terdapat istilah ambigu

In [ ]:
# Fungsi untuk mengecek apakah kata kunci ada dalam teks
def check_keywords(text):
    keywords = [
        'performance', 'space', 'time', 'throughput', 'response', 'memory', 'consumption', 'fast', 'index', 'triggers',
        'storage', 'low', 'run', 'runtime', 'perform', 'execute', 'mean', 'peak', 'compress', 'dynamic', 'offset',
        'reduce', 'fixing', 'early', 'processing', 'security', 'confidentiality', 'integrity', 'availability',
        'accuracy', 'completeness', 'secure', 'access', 'registration', 'authorization', 'identification',
        'authentication', 'validation', 'transaction', 'user', 'password', 'control', 'encryption', 'key', 'spoofing',
        'attack', 'policy', 'logging', 'permission', 'Utility', 'Homogeneity', 'Easiness', 'Operationability',
        'intuitively', 'adaptable', 'Understandability', 'accessible', 'Configuration', 'Integrity', 'Administration',
        'Conformity', 'cognizance', 'applicable', 'Linguistics', 'supportive', 'tutorials', 'trainable', 'helping',
        'flexible', 'easy', 'usable', 'Graphics', 'timing'
    ]
    for keyword in keywords:
        if keyword.lower() in text.lower():
            return 'Non-Functional'
    return 'Functional'

# Menambahkan kolom baru 'Label' berdasarkan analisis menggunakan fungsi check_keywords
data_hasil['Type'] = data_hasil['Context'].apply(check_keywords)

data_hasil

In [ ]:
ambiguous_pronouns_functional = data_hasil[(data_hasil['Pronouns_Ambiguous'] == 1) & (data_hasil['Type'] == 'Functional')]
ambiguous_pronouns_non_functional = data_hasil[(data_hasil['Pronouns_Ambiguous'] == 1) & (data_hasil['Type'] == 'Non-Functional')]

total_ambiguous_pronouns_functional = ambiguous_pronouns_functional.shape[0]
total_ambiguous_pronouns_non_functional = ambiguous_pronouns_non_functional.shape[0]

# Hitung total baris yang memiliki nilai ambigu pada pronouns
total_data_pronouns_ambiguous = data_hasil[data_hasil['Pronouns_Ambiguous'] == 1].shape[0]

percentage_ambiguous_functional = (total_ambiguous_pronouns_functional / total_data_pronouns_ambiguous) * 100
percentage_ambiguous_non_functional = (total_ambiguous_pronouns_non_functional / total_data_pronouns_ambiguous) * 100

print(f"Total Pronouns ambigu: {total_data_pronouns_ambiguous}")
print(f"Functional (Pronouns Ambiguous): {percentage_ambiguous_functional:.2f}%")
print(f"Non-Functional (Pronouns Ambiguous): {percentage_ambiguous_non_functional:.2f}%")

In [ ]:
ambiguous_adj_functional = data_hasil[(data_hasil['ADJ_ADV_Ambiguous'] == 1) & (data_hasil['Type'] == 'Functional')]
ambiguous_adj_non_functional = data_hasil[(data_hasil['ADJ_ADV_Ambiguous'] == 1) & (data_hasil['Type'] == 'Non-Functional')]

total_ambiguous_adj_functional = ambiguous_adj_functional.shape[0]
total_ambiguous_adj_non_functional = ambiguous_adj_non_functional.shape[0]

# Hitung total baris yang memiliki nilai ambigu pada pronouns
total_data_adj_ambiguous = data_hasil[data_hasil['ADJ_ADV_Ambiguous'] == 1].shape[0]

percentage_ambiguous_functional_adj = (total_ambiguous_adj_functional / total_data_adj_ambiguous) * 100
percentage_ambiguous_non_functional_adj = (total_ambiguous_adj_non_functional / total_data_adj_ambiguous) * 100

print(f"Total ADJ_ADV ambigu: {total_data_adj_ambiguous}")
print(f"Functional (ADJ_ADV Ambiguous): {percentage_ambiguous_functional_adj:.2f}%")
print(f"Non-Functional (ADJ_ADV Ambiguous): {percentage_ambiguous_non_functional_adj:.2f}%")

In [ ]:
ambiguous_com_functional = data_hasil[(data_hasil['Comparative_Phrase_Ambiguous'] == 1) & (data_hasil['Type'] == 'Functional')]
ambiguous_com_non_functional = data_hasil[(data_hasil['Comparative_Phrase_Ambiguous'] == 1) & (data_hasil['Type'] == 'Non-Functional')]

total_ambiguous_com_functional = ambiguous_com_functional.shape[0]
total_ambiguous_com_non_functional = ambiguous_com_non_functional.shape[0]

total_data_com_ambiguous = data_hasil[data_hasil['Comparative_Phrase_Ambiguous'] == 1].shape[0]

percentage_ambiguous_functional_com = (total_ambiguous_com_functional / total_data_com_ambiguous) * 100
percentage_ambiguous_non_functional_com = (total_ambiguous_com_non_functional / total_data_com_ambiguous) * 100

print(f"Total Comparative ambigu: {total_data_com_ambiguous}")
print(f"Functional (Comparative Ambiguous): {percentage_ambiguous_functional_com:.2f}%")
print(f"Non-Functional (Comparative Ambiguous): {percentage_ambiguous_non_functional_com:.2f}%")

In [ ]:
ambiguous_label_functional = data_hasil[(data_hasil['Label'] == 1) & (data_hasil['Type'] == 'Functional')]
ambiguous_label_non_functional = data_hasil[(data_hasil['Label'] == 1) & (data_hasil['Type'] == 'Non-Functional')]

total_ambiguous_label_functional = ambiguous_label_functional.shape[0]
total_ambiguous_label_non_functional = ambiguous_label_non_functional.shape[0]

total_data_label_ambiguous = data_hasil[data_hasil['Label'] == 1].shape[0]

percentage_ambiguous_functional_label = (total_ambiguous_label_functional / total_data_label_ambiguous) * 100
percentage_ambiguous_non_functional_label = (total_ambiguous_label_non_functional / total_data_label_ambiguous) * 100

print(f"Total Label ambigu: {total_data_label_ambiguous}")
print(f"Functional (Label Ambiguous): {percentage_ambiguous_functional_label:.2f}%")
print(f"Non-Functional (ADJ_ADV Ambiguous): {percentage_ambiguous_non_functional_label:.2f}%")

## Analisis istilah (Shall, Will dan Should)

In [ ]:
import pandas as pd
import re

# Fungsi untuk mencari kata shall, will, should dalam teks
def find_auxiliary_verbs(text):
    # Menggunakan ekspresi reguler untuk mencari kata shall, will, should dalam teks
    aux_verbs = re.findall(r'\b(?:shall|will|should)\b', text)
    return ', '.join(aux_verbs)

# Membuat kolom baru untuk kata kunci shall, will, should
data_hasil['Auxiliary_Verbs'] = data_hasil['Context'].apply(find_auxiliary_verbs)

# Menampilkan DataFrame dengan kolom baru
data_hasil


### Shall


In [ ]:
# Menghitung jumlah persyaratan yang memenuhi kriteria
shall_pronouns_ambiguous = data_hasil[
    (data_hasil['Auxiliary_Verbs'].str.contains('shall', na=False)) &
    (data_hasil['Pronouns_Ambiguous'] == 1)
].shape[0]

shall_adj_adv_ambiguous = data_hasil[
    (data_hasil['Auxiliary_Verbs'].str.contains('shall', na=False)) &
    (data_hasil['ADJ_ADV_Ambiguous'] == 1)
].shape[0]

shall_comparative_ambiguous = data_hasil[
    (data_hasil['Auxiliary_Verbs'].str.contains('shall', na=False)) &
    (data_hasil['Comparative_Phrase_Ambiguous'] == 1)
].shape[0]

# Menghitung jumlah total persyaratan yang memiliki kata 'shall' di Auxiliary_Verbs
total_requirements_with_shall = data_hasil['Auxiliary_Verbs'].str.contains('shall', na=False).sum()

# Menghitung presentase
percentage_shall_pronouns_ambiguous = (shall_pronouns_ambiguous / total_requirements_with_shall) * 100
percentage_shall_adj_adv_ambiguous = (shall_adj_adv_ambiguous / total_requirements_with_shall) * 100
percentage_shall_comparative_ambiguous = (shall_comparative_ambiguous / total_requirements_with_shall) * 100

# Menampilkan hasil
print(f"Persentase persyaratan dengan kata 'shall' dan Pronouns_Ambiguous label 1: {percentage_shall_pronouns_ambiguous:.2f}%")
print(f"Persentase persyaratan dengan kata 'shall' dan ADJ_ADV_Ambiguous label 1: {percentage_shall_adj_adv_ambiguous:.2f}%")
print(f"Persentase persyaratan dengan kata 'shall' dan Comparative_Phrase_Ambiguous label 1: {percentage_shall_comparative_ambiguous:.2f}%")


In [ ]:
# Menghitung jumlah requirement Non-Functional yang menggunakan kata shall
non_func_shall_count = data_hasil[(data_hasil['Type'] == 'Non-Functional') & (data_hasil['Auxiliary_Verbs'].str.contains('shall'))].shape[0]

# Menghitung jumlah requirement Functional yang menggunakan kata shall
func_shall_count = data_hasil[(data_hasil['Type'] == 'Functional') & (data_hasil['Auxiliary_Verbs'].str.contains('shall'))].shape[0]

# Menghitung total requirement Non-Functional
total_non_func_count = data_hasil[data_hasil['Type'] == 'Non-Functional'].shape[0]

# Menghitung total requirement Functional
total_func_count = data_hasil[data_hasil['Type'] == 'Functional'].shape[0]

# Menghitung presentase requirement Non-Functional yang menggunakan kata shall
percentage_non_func_shall = (non_func_shall_count / total_non_func_count) * 100

# Menghitung presentase requirement Functional yang menggunakan kata shall
percentage_func_shall = (func_shall_count / total_func_count) * 100

print("Total requirement yang menggunakan shall :", total_requirements_with_shall )
print("Presentase Requirement Non-Functional yang menggunakan kata 'shall': {:.2f}%".format(percentage_non_func_shall))
print("Presentase Requirement Functional yang menggunakan kata 'shall': {:.2f}%".format(percentage_func_shall))


### Will

In [ ]:
# Menghitung jumlah persyaratan yang memenuhi kriteria
will_pronouns_ambiguous = data_hasil[
    (data_hasil['Auxiliary_Verbs'].str.contains('will', na=False)) &
    (data_hasil['Pronouns_Ambiguous'] == 1)
].shape[0]

will_adj_adv_ambiguous = data_hasil[
    (data_hasil['Auxiliary_Verbs'].str.contains('will', na=False)) &
    (data_hasil['ADJ_ADV_Ambiguous'] == 1)
].shape[0]

will_comparative_ambiguous = data_hasil[
    (data_hasil['Auxiliary_Verbs'].str.contains('will', na=False)) &
    (data_hasil['Comparative_Phrase_Ambiguous'] == 1)
].shape[0]

# Menghitung jumlah total persyaratan yang memiliki kata 'will' di Auxiliary_Verbs
total_requirements_with_will = data_hasil['Auxiliary_Verbs'].str.contains('will', na=False).sum()

# Menghitung presentase
percentage_will_pronouns_ambiguous = (will_pronouns_ambiguous / total_requirements_with_will) * 100
percentage_will_adj_adv_ambiguous = (will_adj_adv_ambiguous / total_requirements_with_will) * 100
percentage_will_comparative_ambiguous = (will_comparative_ambiguous / total_requirements_with_will) * 100

# Menampilkan hasil
print(f"Persentase persyaratan dengan kata 'will' dan Pronouns_Ambiguous label 1: {percentage_will_pronouns_ambiguous:.2f}%")
print(f"Persentase persyaratan dengan kata 'will' dan ADJ_ADV_Ambiguous label 1: {percentage_will_adj_adv_ambiguous:.2f}%")
print(f"Persentase persyaratan dengan kata 'will' dan Comparative_Phrase_Ambiguous label 1: {percentage_will_comparative_ambiguous:.2f}%")


In [ ]:
# Menghitung jumlah requirement Non-Functional yang menggunakan kata will
non_func_will_count = data_hasil[(data_hasil['Type'] == 'Non-Functional') & (data_hasil['Auxiliary_Verbs'].str.contains('will'))].shape[0]

# Menghitung jumlah requirement Functional yang menggunakan kata will
func_will_count = data_hasil[(data_hasil['Type'] == 'Functional') & (data_hasil['Auxiliary_Verbs'].str.contains('will'))].shape[0]

# Menghitung total requirement Non-Functional
total_non_func_count = data_hasil[data_hasil['Type'] == 'Non-Functional'].shape[0]

# Menghitung total requirement Functional
total_func_count = data_hasil[data_hasil['Type'] == 'Functional'].shape[0]

# Menghitung presentase requirement Non-Functional yang menggunakan kata will
percentage_non_func_will = (non_func_will_count / total_non_func_count) * 100

# Menghitung presentase requirement Functional yang menggunakan kata will
percentage_func_will = (func_will_count / total_func_count) * 100

print("Total requirement yang menggunakan will :", total_requirements_with_will )
print("Presentase Requirement Non-Functional yang menggunakan kata 'will': {:.2f}%".format(percentage_non_func_will))
print("Presentase Requirement Functional yang menggunakan kata 'will': {:.2f}%".format(percentage_func_will))


### Should

In [ ]:
# Menghitung jumlah persyaratan yang memenuhi kriteria
should_pronouns_ambiguous = data_hasil[
    (data_hasil['Auxiliary_Verbs'].str.contains('should', na=False)) &
    (data_hasil['Pronouns_Ambiguous'] == 1)
].shape[0]

should_adj_adv_ambiguous = data_hasil[
    (data_hasil['Auxiliary_Verbs'].str.contains('should', na=False)) &
    (data_hasil['ADJ_ADV_Ambiguous'] == 1)
].shape[0]

should_comparative_ambiguous = data_hasil[
    (data_hasil['Auxiliary_Verbs'].str.contains('should', na=False)) &
    (data_hasil['Comparative_Phrase_Ambiguous'] == 1)
].shape[0]

# Menghitung jumlah total persyaratan yang memiliki kata 'should' di Auxiliary_Verbs
total_requirements_with_should = data_hasil['Auxiliary_Verbs'].str.contains('should', na=False).sum()

# Menghitung presentase
percentage_should_pronouns_ambiguous = (should_pronouns_ambiguous / total_requirements_with_should) * 100
percentage_should_adj_adv_ambiguous = (should_adj_adv_ambiguous / total_requirements_with_should) * 100
percentage_should_comparative_ambiguous = (should_comparative_ambiguous / total_requirements_with_should) * 100

# Menampilkan hasil
print(f"Persentase persyaratan dengan kata 'should' dan Pronouns_Ambiguous label 1: {percentage_should_pronouns_ambiguous:.2f}%")
print(f"Persentase persyaratan dengan kata 'should' dan ADJ_ADV_Ambiguous label 1: {percentage_should_adj_adv_ambiguous:.2f}%")
print(f"Persentase persyaratan dengan kata 'should' dan Comparative_Phrase_Ambiguous label 1: {percentage_should_comparative_ambiguous:.2f}%")


In [ ]:
# Menghitung jumlah requirement Non-Functional yang menggunakan kata should
non_func_should_count = data_hasil[(data_hasil['Type'] == 'Non-Functional') & (data_hasil['Auxiliary_Verbs'].str.contains('should'))].shape[0]

# Menghitung jumlah requirement Functional yang menggunakan kata should
func_should_count = data_hasil[(data_hasil['Type'] == 'Functional') & (data_hasil['Auxiliary_Verbs'].str.contains('should'))].shape[0]

# Menghitung total requirement Non-Functional
total_non_func_count = data_hasil[data_hasil['Type'] == 'Non-Functional'].shape[0]

# Menghitung total requirement Functional
total_func_count = data_hasil[data_hasil['Type'] == 'Functional'].shape[0]

# Menghitung presentase requirement Non-Functional yang menggunakan kata should
percentage_non_func_should = (non_func_should_count / total_non_func_count) * 100

# Menghitung presentase requirement Functional yang menggunakan kata should
percentage_func_should = (func_should_count / total_func_count) * 100

print("Total requirement yang menggunakan should :", total_requirements_with_should )
print("Presentase Requirement Non-Functional yang menggunakan kata 'should': {:.2f}%".format(percentage_non_func_should))
print("Presentase Requirement Functional yang menggunakan kata 'should': {:.2f}%".format(percentage_func_should))



In [ ]:
# Menghitung jumlah requirement Non-Functional atau Functional
count_requirement_type = data_hasil['Label'].value_counts()

# Menampilkan hasil
print(count_requirement_type)


In [ ]:
# Menghitung jumlah requirement yang memiliki label 1 dan menggunakan istilah "shall"
shall_label_1_count = data_hasil[(data_hasil['Label'] == 1) & (data_hasil['Auxiliary_Verbs'].str.contains('shall', case=False))].shape[0]

# Menghitung jumlah requirement yang memiliki label 1 dan menggunakan istilah "will"
will_label_1_count = data_hasil[(data_hasil['Label'] == 1) & (data_hasil['Auxiliary_Verbs'].str.contains('will', case=False))].shape[0]

# Menghitung jumlah requirement yang memiliki label 1 dan menggunakan istilah "should"
should_label_1_count = data_hasil[(data_hasil['Label'] == 1) & (data_hasil['Auxiliary_Verbs'].str.contains('should', case=False))].shape[0]

# Menghitung total requirement yang memiliki label 1
total_label_1_count = data_hasil[data_hasil['Label'] == 1].shape[0]

# Menghitung presentase requirement yang memiliki label 1 dan menggunakan istilah "shall"
percentage_shall_label_1 = (shall_label_1_count / total_label_1_count) * 100

# Menghitung presentase requirement yang memiliki label 1 dan menggunakan istilah "will"
percentage_will_label_1 = (will_label_1_count / total_label_1_count) * 100

# Menghitung presentase requirement yang memiliki label 1 dan menggunakan istilah "should"
percentage_should_label_1 = (should_label_1_count / total_label_1_count) * 100

print(f"Presentase requirement yang memiliki label 1 dan menggunakan istilah 'shall': {percentage_shall_label_1:.2f}%")
print(f"Presentase requirement yang memiliki label 1 dan menggunakan istilah 'will': {percentage_will_label_1:.2f}%")
print(f"Presentase requirement yang memiliki label 1 dan menggunakan istilah 'should': {percentage_should_label_1:.2f}%")


In [ ]:
# Fungsi untuk menghitung jumlah requirement berlabel 1 yang menggunakan kata kunci shall, will, atau should
def count_requirements_with_auxiliary_verbs(df):
    # Filter requirements yang memiliki label 1 dan menggunakan kata kunci shall, will, atau should
    filtered_df = df[(df['Label'] == 1) & (df['Auxiliary_Verbs'] != '')]

    # Menghitung jumlah requirement yang memenuhi kriteria tersebut
    count = len(filtered_df)

    return count

# Memanggil fungsi untuk menghitung jumlah requirement berlabel 1 dengan menggunakan kata kunci shall, will, atau should
jumlah_requirement = count_requirements_with_auxiliary_verbs(data_hasil)

# Menampilkan jumlah requirement yang memenuhi kriteria
print("Jumlah requirement berlabel 1 yang menggunakan kata kunci shall, will, atau should:", jumlah_requirement)


In [ ]:
# Fungsi untuk menghitung presentase requirement berlabel 1 yang menggunakan kata kunci shall, will, atau should
def calculate_percentage_requirements_with_auxiliary_verbs(df):
    # Total jumlah requirement berlabel 1
    total_requirement_label_1 = len(df[df['Label'] == 1])

    # Jumlah requirement berlabel 1 yang menggunakan kata kunci shall, will, atau should
    count_requirement_with_auxiliary_verbs = count_requirements_with_auxiliary_verbs(df)

    # Menghitung persentase
    percentage = (count_requirement_with_auxiliary_verbs / total_requirement_label_1) * 100

    return percentage

# Memanggil fungsi untuk menghitung presentase requirement berlabel 1 yang menggunakan kata kunci shall, will, atau should
presentase_requirement = calculate_percentage_requirements_with_auxiliary_verbs(data_hasil)

# Menampilkan presentase
print("Presentase requirement berlabel 1 yang menggunakan kata kunci shall, will, atau should:", presentase_requirement, "%")


In [ ]:
# Menghitung total jumlah requirement berlabel 1
total_label_1 = data_hasil[data_hasil['Label'] == 1].shape[0]

# Menghitung jumlah requirement berlabel 1 yang tidak menggunakan istilah shall, will, atau should
count_non_aux_verbs = data_hasil[(data_hasil['Label'] == 1) & (~data_hasil['Auxiliary_Verbs'].str.contains(r'shall|will|should', case=False, na=False))].shape[0]

# Menghitung presentasenya
percentage = (count_non_aux_verbs / total_label_1) * 100

print("Presentase requirement berlabel 1 yang tidak menggunakan istilah shall, will, atau should:", percentage, "%")


In [ ]:
# Menghitung jumlah requirement berlabel 1 yang tidak menggunakan istilah shall, will, atau should
count_non_aux_verbs = data_hasil[(data_hasil['Label'] == 1) & (~data_hasil['Auxiliary_Verbs'].str.contains(r'shall|will|should', case=False, na=False))].shape[0]

print("Jumlah requirement berlabel 1 yang tidak menggunakan istilah shall, will, atau should:", count_non_aux_verbs)


In [ ]:
# Fungsi untuk menghitung jumlah requirement berlabel 1 yang menggunakan kata kunci shall, will, atau should
def count_requirements_with_auxiliary_verbs(df):
    # Filter requirements yang memiliki label 1 dan menggunakan kata kunci shall, will, atau should
    filtered_df = df[(df['Label'] == 0) & (df['Auxiliary_Verbs'] != '')]

    # Menghitung jumlah requirement yang memenuhi kriteria tersebut
    count = len(filtered_df)

    return count

# Memanggil fungsi untuk menghitung jumlah requirement berlabel 1 dengan menggunakan kata kunci shall, will, atau should
jumlah_requirement = count_requirements_with_auxiliary_verbs(data_hasil)

# Menampilkan jumlah requirement yang memenuhi kriteria
print("Jumlah requirement berlabel 0 yang menggunakan kata kunci shall, will, atau should:", jumlah_requirement)


In [ ]:
# Fungsi untuk menghitung presentase requirement berlabel 1 yang menggunakan kata kunci shall, will, atau should
def calculate_percentage_requirements_with_auxiliary_verbs(df):
    # Total jumlah requirement berlabel 1
    total_requirement_label_1 = len(df[df['Label'] == 0])

    # Jumlah requirement berlabel 1 yang menggunakan kata kunci shall, will, atau should
    count_requirement_with_auxiliary_verbs = count_requirements_with_auxiliary_verbs(df)

    # Menghitung persentase
    percentage = (count_requirement_with_auxiliary_verbs / total_requirement_label_1) * 100

    return percentage

# Memanggil fungsi untuk menghitung presentase requirement berlabel 1 yang menggunakan kata kunci shall, will, atau should
presentase_requirement = calculate_percentage_requirements_with_auxiliary_verbs(data_hasil)

# Menampilkan presentase
print("Presentase requirement berlabel 0 yang menggunakan kata kunci shall, will, atau should:", presentase_requirement, "%")


In [ ]:
# Menghitung total jumlah requirement berlabel 1
total_label_1 = data_hasil[data_hasil['Label'] == 0].shape[0]

# Menghitung jumlah requirement berlabel 1 yang tidak menggunakan istilah shall, will, atau should
count_non_aux_verbs = data_hasil[(data_hasil['Label'] == 0) & (~data_hasil['Auxiliary_Verbs'].str.contains(r'shall|will|should', case=False, na=False))].shape[0]

# Menghitung presentasenya
percentage = (count_non_aux_verbs / total_label_1) * 100

print("Presentase requirement berlabel 0 yang tidak menggunakan istilah shall, will, atau should:", percentage, "%")


In [ ]:
# Menghitung jumlah requirement berlabel 1 yang tidak menggunakan istilah shall, will, atau should
count_non_aux_verbs = data_hasil[(data_hasil['Label'] == 0) & (~data_hasil['Auxiliary_Verbs'].str.contains(r'shall|will|should', case=False, na=False))].shape[0]

print("Jumlah requirement berlabel 1 yang tidak menggunakan istilah shall, will, atau should:", count_non_aux_verbs)


In [ ]:
def count_label_1(dataframe):
    return dataframe[dataframe['Label'] == 0]['Label'].count()

jumlah_label_1 = count_label_1(data_hasil)
print("Jumlah requirement dengan label 1:", jumlah_label_1)


## Analisis perkembangan frekuensi istilah ambigu pertahun

### 2022


In [ ]:
# Memilih subset dari data yang memiliki Source DAMIR atau DOSSPRE
subset_data = pronoun_ambigu[(pronoun_ambigu['Source'] == 'DAMIR') | (pronoun_ambigu['Source'] == 'DOSSPRE')]

# Menghitung jumlah total konteks pada subset data
total_contexts = len(subset_data)

# Menghitung jumlah konteks yang memiliki label ambigu pada subset data
total_ambiguous_damir = ambiguous_damir_data.shape[0]
total_ambiguous_damir_2 = ambiguous_damir_data_2.shape[0]
total_ambiguous_damir_3 = ambiguous_damir_data_3.shape[0]


total_ambiguous_dosspre = ambiguous_data_dosspre.shape[0]
total_ambiguous_2_dosspre = ambiguous_data_2_dosspre.shape[0]
total_ambiguous_3_dosspre = ambiguous_data_3_dosspre.shape[0]

ambiguous_contexts_1 = total_ambiguous_damir + total_ambiguous_dosspre
ambiguous_contexts_2 = total_ambiguous_damir_2 + total_ambiguous_2_dosspre
ambiguous_contexts_3 = total_ambiguous_damir_3 + total_ambiguous_3_dosspre


# Menghitung persentase konteks yang memiliki label ambigu pada subset data
percentage_ambiguous = (ambiguous_contexts_1 / total_contexts) * 100
percentage_ambiguous_2 = (ambiguous_contexts_2 / total_contexts) * 100
percentage_ambiguous_3 = (ambiguous_contexts_3 / total_contexts) * 100

# Menampilkan hasil
print(f"2022 (Kata Ganti Ambigu): {percentage_ambiguous:.2f}%")
print(f"2022 (Kata Sifat & Keterangan Ambigu): {percentage_ambiguous_2:.2f}%")
print(f"2022 (Frasa Komparatif): {percentage_ambiguous_3:.2f}%")

### 2023

In [ ]:
# Filter dataset untuk mencakup hanya entri 'Source' dan memiliki label ambigu pada kolom Istilah Ambigu
ambiguous_data = data_hasil[(data_hasil['Source'] == 'KAGGLETYPE') & (data_hasil['Pronouns_Ambiguous'] == 1)]
ambiguous_data_2 = data_hasil[(data_hasil['Source'] == 'KAGGLETYPE') & (data_hasil['ADJ_ADV_Ambiguous'] == 1)]
ambiguous_data_3 = data_hasil[(data_hasil['Source'] == 'KAGGLETYPE') & (data_hasil['Comparative_Phrase_Ambiguous'] == 1)]

# Hitung jumlah baris yang memiliki 'Source' DAMIR dan label ambigu pada kolom 'Pronouns_Ambiguous'
total_ambiguous = ambiguous_data.shape[0]
total_ambiguous_2 = ambiguous_data_2.shape[0]
total_ambiguous_3 = ambiguous_data_3.shape[0]

# Hitung total baris yang memiliki 'Source' DAMIR
total_data_kaggle = data_hasil[data_hasil['Source'] == 'KAGGLETYPE'].shape[0]

# Hitung persentase 'Context' yang memiliki label ambigu pada dataset yang memiliki 'Source' DAMIR
percentage_ambiguous = (total_ambiguous / total_data_kaggle) * 100
percentage_ambiguous_2 = (total_ambiguous_2 / total_data_kaggle) * 100
percentage_ambiguous_3 = (total_ambiguous_3 / total_data_kaggle) * 100

print
print(f"KAGGLETYPE (Kata Ganti Ambigu): {percentage_ambiguous:.2f}%")
print(f"KAGGLETYPE (Kata Sifat & Keterangan Ambigu): {percentage_ambiguous_2:.2f}%")
print(f"KAGGLETYPE (Frasa Komparatif Ambigu): {percentage_ambiguous_3:.2f}%")
